Goals
1. Download all zippped files and unzip csv files to 2 seperate folders. One for Ford and one for Lyft - Do in automated fashion if possible
2. Combine each of the 10+ csv files into 2 big files, one for Ford and one for Lyft
3. Push to AWS S3

Bonus: Each month write a code that auto pulls the new zipped file from the cloud and appends it to our existing Lyft AWS dataset

In [9]:
#I will utilize some code from my past web scraping project and tweak it for this project.
#I will use selenium since that is what I have used in the past and is what I am familiar with

#This currently looks up certain fields for you and enters them into the website. 
#Next would be to automate this to run through all 4 designations I want. Do that at end. Also make cleaner later.
import requests
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re
import pandas as pd
from tabulate import tabulate
import os
import math
import regex as re 
from selenium.webdriver.support.ui import Select

from zipfile import ZipFile
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from decimal import Decimal
import numpy as np
import pandas as pd
import math
import os

from datetime import datetime
from datetime import date
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from sklearn.metrics import r2_score
import boto3 


In [21]:
#This will auto download all the code currently on baywheels website

#set download path
chromeOptions = webdriver.ChromeOptions()
prefs = {"download.default_directory" : r"C:\Users\gadde\Desktop\Data_Analytics_projects\baywheels_analysis\zipped_data"}
chromeOptions.add_experimental_option("prefs",prefs)

#start chromedriver
driver = webdriver.Chrome(chrome_options=chromeOptions)
driver.get('https://s3.amazonaws.com/baywheels-data/index.html')
#//*[@id="tbody-content"]/tr[1]/td[1]/a
time.sleep(1)

#Note we have it hard set to 53. Ideally we could use a lenght of something to automatically determine this but I don't
#know how to do that.
for i in range(53):
    download = driver.find_element(by=By.XPATH, value='//*[@id="tbody-content"]/tr[' + str(i+1) + ']/td[1]/a')
    download.click()


C:\Users\gadde\AppData\Local\Temp/ipykernel_10500/3459515471.py:9: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=chromeOptions)


In [32]:
#now let us go ahead and unzip all the files in the zipped_data folder
for root, dirs, files in os.walk('zipped_data'):
    for name in files:
        with ZipFile(os.path.join(root, name), 'r') as zip_ref:
            zip_ref.extractall('baywheels_data\Ford')
            

In [1]:
#I just uploaded one years worth of data to an AWS S3 bucket. Let me check if I can access it here

# Let's use Amazon S3
s3 = boto3.resource("s3")

# Print out bucket names
for bucket in s3.buckets.all():
    print(bucket.name)

baywheels.data


In [4]:
#Now let us see if we can download the data from the s3 bucket
s3 = boto3.client("s3")
s3.download_file(
    Bucket="baywheels.data", Key="2017-fordgobike-tripdata.csv", Filename="aws_data/downloaded_from_s3.csv"
)